In [271]:
import heapq
from math import log2
from tabulate import tabulate

In [272]:
class ShannonFano:
    def __init__(self):
        self.codes = {}

    def encode(self, symbols):
        """Кодирование методом Шеннона-Фано"""
        # Сортируем символы по убыванию вероятностей
        sorted_symbols = sorted(symbols.items(), key=lambda x: x[1], reverse=True)
        self._build_code(sorted_symbols, "")
        return self.codes

    def _build_code(self, symbols, prefix) -> None:
        """Рекурсивное построение кода"""
        if len(symbols) == 1:
            self.codes[symbols[0][0]] = prefix
            return

        # Находим точку разделения
        total_prob = sum(prob for _, prob in symbols)
        half_prob = total_prob / 2

        current_prob = 0
        split_index = 0

        for i, (symbol, prob) in enumerate(symbols):
            current_prob += prob
            if current_prob >= half_prob:
                split_index = i + 1
                break

        # Рекурсивно строим коды для двух групп
        self._build_code(symbols[:split_index], prefix + "0")
        self._build_code(symbols[split_index:], prefix + "1")

In [273]:
class HuffmanNode:
    def __init__(self, symbol=None, prob=0):
        self.symbol = symbol
        self.prob = prob
        self.left = None
        self.right = None

    def __lt__(self, other):
        return self.prob < other.prob

    def __gt__(self, other):
        return self.prob > other.prob

    def __eq__(self, other):
        return self.prob == other.prob

In [274]:
class Huffman:
    def __init__(self):
        self.codes = {}

    def encode(self, symbols):
        """Кодирование методом Хаффмена"""
        # Создаем узлы для каждого символа
        heap = []
        for symbol, prob in symbols.items():
            node = HuffmanNode(symbol, prob)
            heapq.heappush(heap, node)

        # Строим дерево Хаффмена
        while len(heap) > 1:
            left = heapq.heappop(heap)
            right = heapq.heappop(heap)

            merged = HuffmanNode(None, left.prob + right.prob)
            merged.left = left
            merged.right = right

            heapq.heappush(heap, merged)

        # Генерируем коды
        root = heap[0]
        self._generate_codes(root, "")
        return self.codes

    def _generate_codes(self, node, code):
        """Рекурсивная генерация кодов"""
        if node is None:
            return

        if node.symbol is not None:
            self.codes[node.symbol] = code
            return

        self._generate_codes(node.left, code + "0")
        self._generate_codes(node.right, code + "1")

In [275]:
def calculate_metrics(symbols, codes):
    """Вычисление среднего числа символов в коде и избыточности"""
    # Средняя длина кода
    avg_length = 0
    for symbol, prob in symbols.items():
        code_length = len(codes[symbol])
        avg_length += prob * code_length

    # Энтропия
    entropy = 0
    for prob in symbols.values():
        if prob > 0:
            entropy -= prob * log2(prob)

    # Избыточность
    redundancy = avg_length - entropy

    return avg_length, entropy, redundancy

In [276]:
def get_symbols_with_probs(text: str, set_symbols: list[str] | None = None):
    print(set_symbols)
    if set_symbols is None:
        set_symbols = set(text)
    else:
        set_symbols = set(set_symbols)

    symbols_with_count = {}
    symbols_with_probs = {}

    for sym in set_symbols:
        symbols_with_count[sym] = text.count(sym)

    for sym in set_symbols:
        for another_sym in set_symbols:
            if another_sym in sym and sym != another_sym:
                symbols_with_count[sym] -= symbols_with_count[another_sym]

    sum_of_syms = sum(symbols_with_count.values())

    for k in symbols_with_count.keys():
        symbols_with_probs[k] = symbols_with_count[k] / sum_of_syms
    return symbols_with_probs

In [277]:
def build_tree_from_codes(codes):
    """Строит дерево из словаря кодов {символ: двоичный_код}"""
    root = {}

    for symbol, code in codes.items():
        node = root
        for bit in code:
            if bit not in node:
                node[bit] = {}
            node = node[bit]
        node["symbol"] = symbol

    return root

In [278]:
def print_tree(tree, level=0):
    """Рекурсивно печатает дерево"""
    if "symbol" in tree:
        print("  " * level + f"└── [{tree['symbol']}")
        return

    for bit, subtree in tree.items():
        print("  " * level + f"└── {bit}")
        print_tree(subtree, level + 1)

In [279]:
def encode_phrase(phrase: str, codes: dict[str, str]) -> str:
    result = ""
    for i in phrase:
        result += codes[i]
    return result

In [ ]:
def start_programm(
    symbols_with_probs: dict[str, float],
) -> tuple[dict[str, str], dict[str, str]]:
    # print(f"Символы и вероятности:")
    # for symbol, prob in symbols_with_probs.items():
    #     print(f"  {symbol}: {prob:.3f}")

    shannon_fano = ShannonFano()
    sf_codes = shannon_fano.encode(symbols_with_probs)
    sf_avg_length, sf_entropy, sf_redundancy = calculate_metrics(
        symbols_with_probs, sf_codes
    )

    huffman = Huffman()
    hf_codes = huffman.encode(symbols_with_probs)
    hf_avg_length, hf_entropy, hf_redundancy = calculate_metrics(
        symbols_with_probs, hf_codes
    )

    print("Кодирование символов методами Шеннона-Фано и Хаффмена")
    print("=" * 50)

    # Для кодов символов
    table_codes = []
    for symbol in symbols_with_probs:
        table_codes.append(
            [symbol, symbols_with_probs[symbol], sf_codes[symbol], hf_codes[symbol]]
        )

    print(
        tabulate(
            table_codes,
            headers=["Символ", "Вероятность", "Шеннон-Фано", "Хаффмен"],
            tablefmt="grid",
        )
    )

    # Для метрик
    table_metrics = [
        ["Средняя длина", f"{sf_avg_length:.3f}", f"{hf_avg_length:.3f}"],
        ["Энтропия", f"{sf_entropy:.3f}", f"{hf_entropy:.3f}"],
        ["Избыточность", f"{sf_redundancy:.3f}", f"{hf_redundancy:.3f}"],
    ]

    print(
        tabulate(
            table_metrics,
            headers=["Метрика", "Шеннон-Фано", "Хаффмен"],
            tablefmt="grid",
        )
    )

    print("дерево Шеннон-Фано")
    print_tree(build_tree_from_codes(sf_codes))
    print("дерево Хаффмен")
    print_tree(build_tree_from_codes(hf_codes))

    return sf_codes, hf_codes

In [281]:
with open("text") as F:
    TEXT_FROM_FILE = F.read()

TEXT = input(
    "текст из которого будут взяты символы и их вероятность, по умолчанию текст из файла 'text': "
)
if len(TEXT) == 0:
    TEXT = TEXT_FROM_FILE
custom_symbols = ["th", "tion", "ing", "ed", "re", "are", "is", "an", "the", "to"]
use_custom_symbols = input(
    f"использовать символы {custom_symbols}, по умолчанию 0 (1|0): "
)
if use_custom_symbols:
    use_custom_symbols = int(use_custom_symbols)
if not use_custom_symbols:
    custom_symbols = None

symbols_with_probs = get_symbols_with_probs(TEXT, custom_symbols)
if sum(symbols_with_probs.values()) != 1:
    print("WARNING: SUM OF PROBS NOT EQUAL TO 1")

sf_codes, hf_codes = start_programm(symbols_with_probs)
if custom_symbols is None:
    print(f"{TEXT} - фраза исходная")
    print(f"{encode_phrase(TEXT, sf_codes)} фраза закодированная по Шеннон-Фано")
    print(f"{encode_phrase(TEXT, hf_codes)} фраза закодированная по Хаффмену")

None
Кодирование символов методами Шеннона-Фано и Хаффмена
+----------+---------------+---------------+---------------+
| Символ   |   Вероятность |   Шеннон-Фано |       Хаффмен |
+==========+===============+===============+===============+
| C        |   0.000785053 |     111110111 |    0110100011 |
+----------+---------------+---------------+---------------+
| 0        |   0.000314021 |   11111111000 |   01101000010 |
+----------+---------------+---------------+---------------+
| s        |   0.0460041   |         10001 |          0001 |
+----------+---------------+---------------+---------------+
| p        |   0.014602    |       1110001 |        011011 |
+----------+---------------+---------------+---------------+
| l        |   0.0325012   |         10101 |         10101 |
+----------+---------------+---------------+---------------+
| ;        |   0.000471032 |   11111110000 |   01101000011 |
+----------+---------------+---------------+---------------+
| t        |   0.0704977  

In [282]:
symbols_with_probs = dict()

symbols_with_probs = {}

n = int(input("Введите количество символов: "))
for i in range(n):
    symbol = input(f"Символ {i + 1}: ").strip()
    prob = float(input(f"Вероятность для '{symbol}': "))
    symbols_with_probs[symbol] = prob

if sum(symbols_with_probs.values()) != 1:
    print("WARNING: SUM OF PROBS NOT EQUAL TO 1")

sf_codes, hf_codes = start_programm(symbols_with_probs)

Кодирование символов методами Шеннона-Фано и Хаффмена
+----------+---------------+---------------+-----------+
| Символ   |   Вероятность |   Шеннон-Фано |   Хаффмен |
+==========+===============+===============+===========+
| a        |           0.3 |             1 |         0 |
+----------+---------------+---------------+-----------+
| b        |           0.7 |             0 |         1 |
+----------+---------------+---------------+-----------+
+---------------+---------------+-----------+
| Метрика       |   Шеннон-Фано |   Хаффмен |
+===============+===============+===========+
| Средняя длина |         1     |     1     |
+---------------+---------------+-----------+
| Энтропия      |         0.881 |     0.881 |
+---------------+---------------+-----------+
| Избыточность  |         0.119 |     0.119 |
+---------------+---------------+-----------+
дерево Шеннон-Фано
└── 0
  └── [b
└── 1
  └── [a
дерево Хаффмен
└── 0
  └── [a
└── 1
  └── [b
